<a href="https://colab.research.google.com/github/tssriram/web_scraping/blob/main/city_scrape_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import random
import time
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [ ]:
def GET_UA():
    uastrings = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"\
                ]
 
    return random.choice(uastrings)

In [ ]:
URL = "https://www.city-data.com"
page = requests.get(URL, timeout=20, headers={'User-Agent': GET_UA()},verify=False)
soup = BeautifulSoup(page.content, "html.parser")

results = soup.find_all('div',class_='tab-content')
states = re.findall('%s(.*?)%s' % ("cities\">", "</a>"), str(results[0]))

states=pd.DataFrame(states)
states=states[52:102].reset_index(drop=True)
states[0]=states[0].str.strip()
states[0]=states[0].str.replace(' ','-')

In [ ]:
data=pd.DataFrame()

for j in states.iloc[:,0]:  

  URL = "https://www.city-data.com/city/{}.html"
  page=''
  while page=='':
    try:
      page = requests.get(URL.format(j), timeout=20, headers={'User-Agent': GET_UA()},verify=False)
      break
    except:
      print('\n SLEEP \n')
      time.sleep(5)
      continue

  soup = BeautifulSoup(page.content, "html.parser")

  results = soup.find_all('table',class_='tabBlue tblsort tblsticky')
  tdf=pd.read_html(str(results))[0]
  tdf=tdf.iloc[:,1:3]
  tdf=tdf[tdf.Population>50000]
  if len(tdf)==0:
    continue
  tdf.Name=[p.split(',',1)[0] for p in tdf.Name]
  tdf.Name=tdf.Name.str.strip()
  tdf.Name=tdf.Name.str.replace(' ','-')
  tdf.Name=tdf.Name+'-'+j

  city=pd.DataFrame()
  print(j)

  for i in tqdm(tdf.Name):
    URL="https://www.city-data.com/city/{}.html"
    headers = {'User-Agent': GET_UA()}
    
    page=''
    while page=='':
      try:
        page = requests.get(URL.format(i), timeout=20, headers={'User-Agent': GET_UA()},verify=False)
        break
      except:
        print('Breaker')
        time.sleep(5)
        continue

    soup = BeautifulSoup(page.content, "html.parser")

    #print(i)

    #population
    try:
      pop = soup.find_all('section',class_='city-population',id="city-population")
      cpop = re.search('</b>(.*?)<', str(pop[0]))
      cpopy = re.search('<b>(.*?)</b>', str(pop[0]))
      city_population=cpop.group(1).strip()
      city_population_year=re.sub("[^0-9]", "",cpopy.group(1))
    except:
      city_population=''
      city_population_year=''


    #sex
    try:
      popsex = soup.find_all('section',class_='population-by-sex',id="population-by-sex")
      male_pop_sex = re.search('%s(.*?)%s' % ("\(", "\)"), str(popsex[0]) ).group(1)
    except:
      male_pop_sex=''

    #age
    try:
      medage = soup.find_all('section',class_='median-age',id="median-age")
      median_age = re.search('%s(.*?)%s' % ("</p>","</td>"), str(medage[0]) ).group(1)
    except:
      median_age=''

    #wealth
    try:
      medinc = soup.find_all('section',class_='median-income',id="median-income")
      wealth = re.findall('%s(.*?)%s' % ("</b> "," \("), str(medinc[0]))
      median_household_income = wealth[0]
      per_capita_income = wealth[1]
      house_value = wealth[2]
    except:
      median_household_income = ''
      per_capita_income = ''
      house_value = ''

    #rent
    try:
      rent = soup.find_all('section',class_='median-rent',id="median-rent")
      median_rent = re.findall('%s(.*?)%s' % ("</b> ","\."), str(rent[0]))[0]
    except:
      median_rent = ''

    #cost of living
    try:
      col = soup.find_all('section',class_='cost-of-living-index',id="cost-of-living-index")
      cost_of_living = re.findall('%s(.*?)%s' % ("</b> "," "), str(col[0]))[0]
    except:
      cost_of_living=''

    #poverty level
    try:
      pov = soup.find_all('section',class_='poverty-level',id="poverty-level")
      poverty_level = re.findall('%s(.*?)%s' % ("</b> ","<br/>"), str(pov[0]))[0]
    except:
      poverty_level=''

    #crime
    try:
      crime = soup.find_all('section',class_='crime',id="crime")
      crime_index=pd.read_html(str(crime))[0]
      crime_index = crime_index.iloc[crime_index.shape[0]-1,crime_index.shape[1]-1]
    except:
      crime_index=''

    #population density
    try:
      popd = soup.find_all('section',class_='population-density',id="population-density")
      population_density = re.findall('%s(.*?)%s' % ("</b> "," <b>"), str(popd[0]))[1]
    except:
      population_density=''

    #unemployment
    try:
      unemp = soup.find_all('section',class_='unemployment',id="unemployment")
      unemployment = re.findall('%s(.*?)%s' % ("</p>","</td>"), str(unemp[0]))[0]
    except:
      unemployment=''
    
    row=pd.DataFrame([i,city_population_year,city_population,male_pop_sex,median_age,median_household_income,per_capita_income,house_value,median_rent,cost_of_living,poverty_level,crime_index,population_density,unemployment]).transpose()
    row.columns=['city','data_year','population','percent_male','median_age','median_household_income','per_capita_income','median_house_value','median_rent','cost_of_living','poverty-level','crime_index','population_density_sqmile','unemployment']

    city=city.append(row)

  data=data.append(city)

Alabama


100%|██████████| 10/10 [00:23<00:00,  2.37s/it]


Alaska


100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


Arkansas


...

Breaker


 74%|███████▍  | 158/214 [07:47<12:11, 13.07s/it]

Breaker


 74%|███████▍  | 158/214 [07:52<02:47,  2.99s/it]


KeyboardInterrupt: ignored

In [ ]:
data

,city,data_year,population,percent_male,median_age,median_household_income,per_capita_income,median_house_value,median_rent,cost_of_living,poverty-level,crime_index,population_density_sqmile,unemployment
0,Auburn-Alabama,2019,"66,259 (93% urban, 7% rural).",50.8%,25.2 years,"$46,031","$30,979","$304,000",$770,85.5,26.1%,198.2,"1,693",2.9%
0,Birmingham-Alabama,2019,"209,403 (99% urban, 1% rural).",47.1%,36.7 years,"$36,753","$25,552","$98,800",$839,86.6,25.2%,875.8,"1,397",6.8%
0,Decatur-Alabama,2019,"54,445 (95% urban, 5% rural).",47.8%,41.6 years,"$50,455","$30,080","$148,834",$712,81.5,17.5%,340.0,"1,019",3.8%
0,Dothan-Alabama,2019,"68,941 (94% urban, 6% rural).",48.2%,41.0 years,"$47,411","$29,852","$162,200",$734,80.7,20.1%,453.9,796,4.7%
0,Hoover-Alabama,2019,"85,768 (100% urban, 0% rural).",47.5%,36.9 years,"$98,689","$46,896","$322,000","$1,153",88.4,5.2%,147.6,"1,988",2.5%
0,Huntsville-Alabama,2019,"200,574 (97% urban, 3% rural).",48.0%,35.8 years,"$54,342","$34,274","$185,200",$846,83.4,17.5%,505.2,"1,152",3.8%
0,Mobile-Alabama,2019,"188,720 (99% urban, 1% rural).",46.6%,37.5 years,"$46,788","$27,432","$129,700",$876,83.5,17.6%,453.1,"1,601",6.9%
0,Montgomery-Alabama,2019,"198,525 (98% urban, 2% rural).",47.2%,35.6 years,"$51,074","$27,861","$125,000",$875,84.3,16.1%,401.5,"1,278",6.6%
0,Tanner-Williams-Alabama,2007,"60,211.",48.3%,32.2 years,"$63,588","$31,562","$177,507",,88.1,,,557,5.8%
0,Tuscaloosa-Alabama,2019,"101,129 (99% urban, 1% rural).",49.3%,31.0 years,"$40,934","$28,498","$200,200",$886,87.2,20.1%,359.4,"1,798",5.1%
